In [223]:
import numpy as np
import pandas as pd

path='../raw/'
date_list=['2021-04-19','2021-04-20','2021-04-21','2021-05-05','2021-05-06']

meta=pd.read_csv('../meta/Hwy20_detectors.csv')



In [224]:
meta[meta['Intersection']=='Century']['Detector'].unique()

array([35., 33., 34., 49., 50., 51., 52., 63., 39., 37., 53., 54., 43.,
       41., 44., 57., 58., 59., 60., 64., 47., 45., 61., 62.])

In [225]:
meta=meta[['Intersection','Phase','Detector','Detection Type','Detection Hardware']].drop_duplicates()
meta['Detection Type']=np.where(meta['Detection Type']=='Normal','Stop Bar Presence','Lane-by-lane Count')
meta[meta['Intersection']=='Century']['Detection Type'].unique()

array(['Stop Bar Presence', 'Lane-by-lane Count'], dtype=object)

In [226]:
manual=pd.DataFrame([])
for d in date_list:

    occupancy_of_chunk_150_lenght=pd.read_csv('../occupancy/'+d+'/'+'occupancy_of_chunk_150_lenght'+d+'.csv', index_col=[0])


    occupancy_of_chunk_150_lenght=occupancy_of_chunk_150_lenght.sort_values(['signal','detector_parameter','unique_cycle_id','chunk'])
    occupancy_of_chunk_150_lenght['below_second']=occupancy_of_chunk_150_lenght.groupby(['signal','detector_parameter','unique_cycle_id'])['seconds'].shift(-1)

    occupancy_of_chunk_150_lenght['point_difference']=occupancy_of_chunk_150_lenght['below_second']-occupancy_of_chunk_150_lenght['seconds']
    occupancy_of_chunk_150_lenght.start_cycle=pd.to_datetime(occupancy_of_chunk_150_lenght.start_cycle)
    occupancy_of_chunk_150_lenght['date']=occupancy_of_chunk_150_lenght.start_cycle.dt.date


    temp=occupancy_of_chunk_150_lenght[['signal','date','detector_parameter','unique_cycle_id','chunk','point_difference']]
    temp=temp[temp.point_difference<0]

    temp=temp.sort_values(['signal','date','detector_parameter','unique_cycle_id','chunk']).assign(
    first_ocup_drop_chunk = 
    temp
    .groupby(['signal','date','detector_parameter','unique_cycle_id'])
    .chunk
    .transform('min')
    )

    temp=temp.assign(
    max_occup_drop_value = 
    temp
    .groupby(['signal','date','detector_parameter','unique_cycle_id'])
    .point_difference
    .transform('min')
    )


    temp['drop_point']=np.where(temp.max_occup_drop_value==temp.point_difference, 'maximum_drop_point','na' )


    #alternative_solution_phase=pd.read_csv('../../phase_prob/'+d+'/'+'adjustment_left_merged_1'+d+'.csv', index_col=[0])
    alternative_solution_phase=pd.read_csv('../phase_prob/'+d+'/'+'alternative_solution_phase_probability'+d+'.csv', index_col=[0])

    alternative_solution_phase.cycle_start=pd.to_datetime(alternative_solution_phase.cycle_start)
    alternative_solution_phase['date']=alternative_solution_phase.cycle_start.dt.date


    temp2=alternative_solution_phase[(alternative_solution_phase['event_color']=='Green') & (alternative_solution_phase['location']=='first') ].sort_values(['signal','parameter','unique_cycle_id','b']).groupby(['signal','parameter','unique_cycle_id']).first().reset_index()
    temp2=temp2[['signal', 'date','parameter', 'cycle_id', 'unique_cycle_id', 'event_color',
       'location', 'b',  'below_chunk', 'event_start_time',
       'event_end_time', 'cycle_start', 'cycle_end', 'event', 'chunk_start',
       'chunk_end']]


    combination=temp2.merge(temp, how='left', on=['signal','unique_cycle_id','date'])[['signal','date','parameter','detector_parameter','cycle_id','unique_cycle_id','chunk','b','first_ocup_drop_chunk','max_occup_drop_value','drop_point']]
    combination['chunk_difference']=combination['chunk']-combination['b']
    combination=combination[( combination['chunk_difference']<=2) & ( combination['chunk_difference']>=0) ].rename(columns={'chunk':'occupancy_drop_chunk','b':'phase_first_green_chunk'})
    combination=combination.reset_index(drop=True)
    combination['drop_id']=combination.index

    combination=combination.merge(meta, left_on=['signal','detector_parameter'], right_on=['Intersection','Detector'], how='left')
    method1=combination.groupby(['signal','date','detector_parameter','Phase','parameter','chunk_difference'])['drop_id'].count().reset_index()
    method1=method1.rename(columns={'drop_id':'count_of_drop'})

    method2=method1.assign(
        max_count_drop = 
        method1
        .groupby(['signal','date','detector_parameter','Phase','chunk_difference'])
        .count_of_drop
        .transform('max')
        )

    method3=method2[method2['max_count_drop']==method2['count_of_drop']].sort_values(['signal','date','detector_parameter','Phase','chunk_difference','max_count_drop'])

    method4=method3.assign(
    frequent_parameter_count= 
    method3
    .groupby(['signal','date','detector_parameter','parameter'])
    .chunk_difference
    .transform('count')
    )



    #method4=method3.groupby(['signal','detector_parameter','parameter'])['chunk_difference'].count().reset_index()
    method5=method4.drop_duplicates(['signal','date','detector_parameter','Phase','parameter']).sort_values(['signal','detector_parameter','Phase','parameter'])

    method6=method5[['signal','date','detector_parameter','Phase','parameter','frequent_parameter_count']].assign(
    max_of_frequent_parameter_count= 
    method5
    .groupby(['signal','date','detector_parameter','Phase'])
    .frequent_parameter_count
    .transform('max')
    )

    method7=method6[method6['frequent_parameter_count']==method6['max_of_frequent_parameter_count']]
    manual=manual.append(method7)

In [227]:
manual['correct']='na'
manual['correct']=np.where(manual['Phase']==manual['parameter'],1,0)

In [228]:
manual.to_csv('manual.csv')

In [70]:
manual=pd.read_csv('manual.csv', index_col=[0])

In [229]:
summary=manual.groupby(['signal','detector_parameter','parameter','Phase'])['correct'].count().reset_index().rename(columns={'correct':'count'})
summary.sort_values(['signal','detector_parameter','parameter','count'])

,signal,detector_parameter,parameter,Phase,count
0,Century,33.0,2.0,2.0,2
1,Century,33.0,6.0,2.0,3
2,Century,35.0,2.0,5.0,2
3,Century,35.0,5.0,5.0,5
4,Century,37.0,4.0,4.0,5
...,...,...,...,...,...
291,Wacker,45.0,4.0,8.0,3
292,Wacker,45.0,5.0,8.0,2
293,Wacker,45.0,8.0,8.0,4
294,Wacker,47.0,3.0,3.0,5


In [230]:
summary=summary.merge(meta, left_on=['signal','detector_parameter','Phase'], right_on=['Intersection','Detector','Phase'])
summary['Detection Type'].unique()

array(['Stop Bar Presence', 'Lane-by-lane Count'], dtype=object)

In [231]:
result=summary.loc[summary.groupby(['signal','detector_parameter'])['count'].idxmax()]
result['correct']=np.where(result['parameter']==result['Phase'],1,0)
result

,signal,detector_parameter,parameter,Phase,count,Intersection,Detector,Detection Type,Detection Hardware,correct
1,Century,33.0,6.0,2.0,3,Century,33.0,Stop Bar Presence,Matrix,0
3,Century,35.0,5.0,5.0,5,Century,35.0,Stop Bar Presence,Matrix,1
4,Century,37.0,4.0,4.0,5,Century,37.0,Stop Bar Presence,Matrix,1
6,Century,39.0,7.0,7.0,5,Century,39.0,Stop Bar Presence,Matrix,1
7,Century,41.0,1.0,6.0,5,Century,41.0,Stop Bar Presence,Matrix,0
...,...,...,...,...,...,...,...,...,...,...
285,Wacker,39.0,3.0,7.0,5,Wacker,39.0,Stop Bar Presence,Matrix,0
290,Wacker,41.0,6.0,6.0,4,Wacker,41.0,Stop Bar Presence,Matrix,1
291,Wacker,43.0,1.0,1.0,5,Wacker,43.0,Stop Bar Presence,Matrix,1
296,Wacker,45.0,8.0,8.0,4,Wacker,45.0,Stop Bar Presence,Matrix,1


In [232]:
result=result[result['Phase']<9]
#result=result[result['Detection Hardware']!='Advanced']

In [233]:
result=result[~result['Detection Hardware'].isin(['Advanced ','Advanced'])]
result['Detection Hardware'].unique()

array(['Matrix'], dtype=object)

In [234]:
result['Detection Type'].unique()

array(['Stop Bar Presence', 'Lane-by-lane Count'], dtype=object)

In [237]:
result.to_csv('result_1.csv')

In [236]:
g=result.groupby('signal')
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets, 'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy

,signal,accuracy,number of detectors
0,Century,0.700000,20
0,Crescent Ridge,0.666667,18
0,Devon,0.476190,21
0,JFK,0.500000,22
0,NW Arterial,0.681818,22
0,University,0.578947,19
0,Wacker,0.750000,8


# after removing the low data and now signal channels

In [4]:
import pandas as pd
result3=pd.read_csv('result_3.csv')
g=result3.groupby('signal')
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets, 'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy
#century signal is with left_merged_   for signal color the following cell
#Devon signal as well

,signal,accuracy,number of detectors
0,Century,0.875000,16
0,Crescent Ridge,0.923077,13
0,Devon,0.588235,17
0,JFK,0.785714,14
0,NW Arterial,0.789474,19
0,University,0.687500,16
0,Wacker,0.750000,8


In [212]:
g=result.groupby('signal')
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets, 'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy

,signal,accuracy,number of detectors
0,Century,0.900000,20
0,Crescent Ridge,0.666667,18
0,Devon,0.619048,21
0,JFK,0.545455,22
0,NW Arterial,0.681818,22
0,University,0.526316,19
0,Wacker,0.750000,8


# skewsness

In [240]:
type_def=occupancy_of_chunk_150_lenght[['signal','detector_parameter','Phase','Detection Type']].drop_duplicates()

In [241]:
import scipy
from scipy.stats import skew
t=occupancy_of_chunk_150_lenght.groupby(['signal','detector_parameter','Phase'])['seconds'].agg(scipy.stats.skew).reset_index()



t=t.merge(type_def)
t=t.dropna()
t=t.rename({'seconds':'occupancy_skewness'}, axis=1)

In [242]:
t

,signal,detector_parameter,Phase,occupancy_skewness,Detection Type
0,Century,33.0,2.0,-1.523892,Stop Bar Presence
1,Century,35.0,5.0,-3.936130,Stop Bar Presence
2,Century,37.0,4.0,-2.591769,Stop Bar Presence
3,Century,39.0,7.0,-2.883627,Stop Bar Presence
4,Century,41.0,6.0,-1.357221,Stop Bar Presence
...,...,...,...,...,...
145,Wacker,39.0,7.0,-2.873075,Stop Bar Presence
146,Wacker,41.0,6.0,-2.504081,Stop Bar Presence
147,Wacker,43.0,1.0,-4.348838,Stop Bar Presence
148,Wacker,45.0,8.0,-4.318215,Stop Bar Presence


In [245]:
skewssnes_hist=occupancy_of_chunk_150_lenght.merge(t, how='left', on=['signal','detector_parameter', 'Phase'])
skewssnes_hist

,signal,detector_parameter,cycle_id,unique_cycle_id,start_cycle,end_cycle,chunk,seconds,Intersection,Phase,Detector,Detection Type_x,Detection Hardware,below_second,point_difference,date,occupancy_skewness,Detection Type_y
0,Century,33.0,0,0,2021-05-06 14:37:26.600,2021-05-06 14:39:36.600,1.0,0.4,Century,2.0,33.0,Stop Bar Presence,Matrix,4.0,3.6,2021-05-06,-1.523892,Stop Bar Presence
1,Century,33.0,0,0,2021-05-06 14:37:26.600,2021-05-06 14:39:36.600,2.0,4.0,Century,2.0,33.0,Stop Bar Presence,Matrix,5.0,1.0,2021-05-06,-1.523892,Stop Bar Presence
2,Century,33.0,0,0,2021-05-06 14:37:26.600,2021-05-06 14:39:36.600,3.0,5.0,Century,2.0,33.0,Stop Bar Presence,Matrix,5.0,0.0,2021-05-06,-1.523892,Stop Bar Presence
3,Century,33.0,0,0,2021-05-06 14:37:26.600,2021-05-06 14:39:36.600,4.0,5.0,Century,2.0,33.0,Stop Bar Presence,Matrix,5.0,0.0,2021-05-06,-1.523892,Stop Bar Presence
4,Century,33.0,0,0,2021-05-06 14:37:26.600,2021-05-06 14:39:36.600,5.0,5.0,Century,2.0,33.0,Stop Bar Presence,Matrix,4.1,-0.9,2021-05-06,-1.523892,Stop Bar Presence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96994,Wacker,47.0,46,337,2021-05-06 18:01:23.700,2021-05-06 18:03:33.700,14.0,5.0,Wacker,3.0,47.0,Stop Bar Presence,Matrix,5.0,0.0,2021-05-06,-2.672002,Stop Bar Presence
96995,Wacker,47.0,46,337,2021-05-06 18:01:23.700,2021-05-06 18:03:33.700,15.0,5.0,Wacker,3.0,47.0,Stop Bar Presence,Matrix,2.1,-2.9,2021-05-06,-2.672002,Stop Bar Presence
96996,Wacker,47.0,46,337,2021-05-06 18:01:23.700,2021-05-06 18:03:33.700,16.0,2.1,Wacker,3.0,47.0,Stop Bar Presence,Matrix,4.5,2.4,2021-05-06,-2.672002,Stop Bar Presence
96997,Wacker,47.0,46,337,2021-05-06 18:01:23.700,2021-05-06 18:03:33.700,25.0,4.5,Wacker,3.0,47.0,Stop Bar Presence,Matrix,5.0,0.5,2021-05-06,-2.672002,Stop Bar Presence


In [92]:
skewssnes_hist.to_csv('skewness_histogram.csv')

In [ ]:
#frequency skewness

In [50]:
frequency=pd.read_csv('C:/Users/atousaz/Box/Sharma/1week/foothill/frequency/2021-04-16/frequency_v1_2021-04-16.csv', index_col=[0])
laf=frequency.groupby(['signal','detector_parameter','Phase'])['count'].agg(scipy.stats.skew).reset_index()
laf=laf.merge(type_def)
laf=laf.dropna()
laf

,signal,detector_parameter,Phase,count,DetectionTypes
0,7216,10,2.0,1.800929,Advanced Count
1,7216,12,6.0,1.138672,Advanced Count
2,7216,17,5.0,0.000000,Stop Bar Presence
3,7216,18,2.0,3.152956,Stop Bar Presence
4,7216,19,2.0,2.598004,Stop Bar Presence
...,...,...,...,...,...
180,7371,51,8.0,2.418679,Lane-by-lane Count
181,7503,2,2.0,1.364285,Advanced Count
182,7503,4,6.0,1.084341,Advanced Count
183,7503,5,4.0,5.467041,Stop Bar Presence


In [51]:
laf.to_csv('frequency_skewness.csv')

In [54]:
sfc=frequency.groupby(['signal','detector_parameter','Phase'])['count'].agg(sum).reset_index()
sfc=sfc.merge(type_def)
sfc=sfc.dropna()
sfc

,signal,detector_parameter,Phase,count,DetectionTypes
0,7216,10,2.0,1755,Advanced Count
1,7216,12,6.0,3802,Advanced Count
2,7216,17,5.0,78,Stop Bar Presence
3,7216,18,2.0,558,Stop Bar Presence
4,7216,19,2.0,755,Stop Bar Presence
...,...,...,...,...,...
180,7371,51,8.0,345,Lane-by-lane Count
181,7503,2,2.0,4042,Advanced Count
182,7503,4,6.0,5608,Advanced Count
183,7503,5,4.0,237,Stop Bar Presence


In [68]:
#  add deviation to occupancy skewness
std=frequency.groupby(['signal','detector_parameter','Phase'])['count'].agg(np.std).reset_index()

std=std.rename({'count':'frequency_sd'},axis=1)
ostd=occupancy_of_chunk_150_lenght.groupby(['signal','detector_parameter','Phase'])['seconds'].agg(np.std).reset_index()
ostd=ostd.rename({'seconds':'occupancy_sd'},axis=1)

bsa=t.merge(sfc, how='left').merge(ostd).merge(std).rename({'count':'sum_count','seconds':'occupancy_skewness'},axis=1)
bsa.to_csv('skewness.csv')

In [46]:
t.to_csv('skewness.csv')

In [246]:
#t=pd.read_csv('skewness.csv', index_col=[0])

In [96]:
t['type']=np.where(t.occupancy_skewness>0, 'Lane-by-lane Count','Stop Bar Presence')

In [250]:
t['Detection Type'].unique()

array(['Stop Bar Presence', 'Lane-by-lane Count'], dtype=object)

In [252]:
#t=t[t['DetectionTypes']!='Advanced Count']
t

,signal,detector_parameter,Phase,occupancy_skewness,Detection Type
0,Century,33.0,2.0,-1.523892,Stop Bar Presence
1,Century,35.0,5.0,-3.936130,Stop Bar Presence
2,Century,37.0,4.0,-2.591769,Stop Bar Presence
3,Century,39.0,7.0,-2.883627,Stop Bar Presence
4,Century,41.0,6.0,-1.357221,Stop Bar Presence
...,...,...,...,...,...
145,Wacker,39.0,7.0,-2.873075,Stop Bar Presence
146,Wacker,41.0,6.0,-2.504081,Stop Bar Presence
147,Wacker,43.0,1.0,-4.348838,Stop Bar Presence
148,Wacker,45.0,8.0,-4.318215,Stop Bar Presence


In [251]:
t=t.dropna()

In [253]:
t['type']=np.where(t['occupancy_skewness']>0,'Lane-by-lane Count','Stop Bar Presence')

In [254]:
t['correct']=np.where(t['Detection Type']==t['type'],1,0)

In [255]:
g=t.groupby('signal')
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets, 'accuracy':accuracy,'number of detectors':len(data)}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy

,signal,accuracy,number of detectors
0,Century,0.913043,23
0,Crescent Ridge,0.952381,21
0,Devon,1.000000,24
0,JFK,0.920000,25
0,NW Arterial,0.961538,26
0,University,0.956522,23
0,Wacker,1.000000,8


In [ ]:
tt=t.merge(occupancy_of_chunk_120_lenght4[['signal','detector_parameter','Description_y']].drop_duplicates())
tt['correct']=np.where(tt['type']==tt['Description_y'],1,0)

In [ ]:
g=tt.groupby('signal')
detailed_accuracy=pd.DataFrame([])
for sets, data in g:
    accuracy=len(data[data['correct']==1])/len(data)
    df=pd.DataFrame({'signal':sets, 'accuracy':accuracy}, index=[0])
    detailed_accuracy=detailed_accuracy.append(df)
detailed_accuracy